# JAX for the impatient
**JAX is NumPy on the CPU, GPU, and TPU, with great automatic differentiation for high-performance machine learning research.**

Here we will cover the basics of JAX so that you can get started with Flax, however we very much recommend that you go through JAX's documentation [here](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html) after going over the basics here.


## NumPy API
Let's start by exploring the NumPy API coming from JAX and the main differences you should be aware of.

In [1]:
import jax
from jax import numpy as jnp, random

import numpy as np # We import the standard NumPy library 

`jax.numpy` is the NumPy-like API that needs to be imported, and we will also use `jax.random` to generate some data to work on. 

Let's start by generating some matrices, and then try matrix multiplication.

In [2]:
m = jnp.ones((4,4)) # We're generating one 4 by 4 matrix filled with ones.
n = jnp.array([[1.0, 2.0, 3.0, 4.0],
               [5.0, 6.0, 7.0, 8.0]]) # An explicit 2 by 4 array
m

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


DeviceArray([[1., 1., 1., 1.],
             [1., 1., 1., 1.],
             [1., 1., 1., 1.],
             [1., 1., 1., 1.]], dtype=float32)

Arrays in JAX are represented as DeviceArray instances and are agnostic to the place where the array lives (CPU, GPU, or TPU). This is why we're getting the warning that no GPU/TPU was found and JAX is falling back to CPU (unless you're running it in an environment that has a GPU/TPU available).

We can obviously multiply matrices like we would do in NumPy.

In [3]:
jnp.dot(n, m).block_until_ready() # Note: yields the same result as n.dot(m)

DeviceArray([[10., 10., 10., 10.],
             [26., 26., 26., 26.]], dtype=float32)

DeviceArray instances are actually futures ([more here](https://jax.readthedocs.io/en/latest/async_dispatch.html)) due to the **default asynchronous execution** in JAX. For that reason, the Python call might return before the computation actually ends, hence we're using the `block_until_ready()` method to ensure we return the end result.

JAX is fully compatible with NumPy, and can transparently process arrays from one library to the other.

In [4]:
x = np.random.normal(size=(4,4)) # Creating one standard NumPy array instance
jnp.dot(x,m)

DeviceArray([[-2.1363673 , -2.1363673 , -2.1363673 , -2.1363673 ],
             [ 2.5145242 ,  2.5145242 ,  2.5145242 ,  2.5145242 ],
             [-0.41032386, -0.41032386, -0.41032386, -0.41032386],
             [ 0.29587948,  0.29587948,  0.29587948,  0.29587948]],            dtype=float32)

If you're using accelerators, using NumPy arrays directly will result in multiple transfers from CPU to GPU/TPU memory. You can save that transfer bandwidth, either by creating directly a DeviceArray or by using `jax.device_put` on the NumPy array.

In [5]:
x = np.random.normal(size=(4,4))
x = jax.device_put(x)
x

DeviceArray([[-2.6163752 ,  0.44878265,  1.5077958 ,  1.4197438 ],
             [-0.8818235 ,  0.89041513,  0.8823177 , -0.50059307],
             [ 0.8309847 , -0.6925419 ,  0.2270658 ,  1.0040795 ],
             [ 1.1299754 , -0.09446267,  0.4083803 ,  0.8279651 ]],            dtype=float32)

Conversely, if you want to get back a Numpy array from a JAX array, you can simply do so by using it in the Numpy API.

In [6]:
x = jnp.array([[1.0, 2.0, 3.0, 4.0],
               [5.0, 6.0, 7.0, 8.0]])
np.array(x)

array([[1., 2., 3., 4.],
       [5., 6., 7., 8.]], dtype=float32)

## (Im)mutability in JAX
JAX is functional by essence, one practical consequence being that JAX arrays are immutable. This means no in-place ops and sliced assignments. More generally, functions should not take input or produce output using a global state.



In [7]:
x = jnp.array([[1.0, 2.0, 3.0, 4.0],
               [5.0, 6.0, 7.0, 8.0]])
jax.ops.index_update(x,(0,0), 3.0) # whereas x[0,0] = 3.0 would fail

DeviceArray([[3., 2., 3., 4.],
             [5., 6., 7., 8.]], dtype=float32)

Index operators can be found in `jax.ops`and follow the `index_*` pattern. To generate index in those syntax, one can use the `jax.ops.index` syntactic sugar.

In [ ]:
x = jnp.array([[1.0, 2.0, 3.0, 4.0],
               [5.0, 6.0, 7.0, 8.0]])
jax.ops.index_update(x,jax.ops.index[0,:], 3.0) # Same as x[O,:] = 3.0 in NumPy.

DeviceArray([[3., 3., 3., 3.],
             [5., 6., 7., 8.]], dtype=float32)

Finally, a more concise (and modern) way would be to use the `.at` attribute that plays the same syntactic sugar role as previously:

In [8]:
x.at()
x.at[0,:].set(3.0) # Note: this returns a new array and doesn't mutate in place.

DeviceArray([[3., 3., 3., 3.],
             [5., 6., 7., 8.]], dtype=float32)

All jax ops are available with this syntax, including: `set`, `add`, `mul`, `min`, `max`.

## Managing randomness in JAX
In JAX, randomness is managed in a very specific way, and you can read more on JAX's docs [here](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#%F0%9F%94%AA-Random-Numbers) (we borrow content from there!). As the JAX team puts it:

*JAX implements an explicit PRNG where entropy production and consumption are handled by explicitly passing and iterating PRNG state. JAX uses a modern Threefry counter-based PRNG that’s splittable. That is, its design allows us to fork the PRNG state into new PRNGs for use with parallel stochastic generation.*

In short, you need to explicitely manage the PRNGs (pseudo random number generators) and their states. In JAX's PRNGs, the state is represented as a pair of two unsigned-int32s that is called a key.

In [ ]:
key = random.PRNGKey(0)
key

DeviceArray([0, 0], dtype=uint32)

If you use this key multiple times, you will not get any randomness in the output. To generate further entries in the sequence, you'll need to split the PRNG and thus generate a new pair of keys.

In [ ]:
print("old key", key)
key, subkey = random.split(key)
normal_pseudorandom = random.normal(subkey, shape=(1,)) # We explicitely provide the PRNG as an input to the random matrix generator.
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

old key [0 0]
    \---SPLIT --> new key    [4146024105  967050713]
             \--> new subkey [2718843009 1272950319] --> normal [-1.2515389]


You can also generate multiple subkeys at once if needed:

In [ ]:
key, *subkeys = random.split(key, 4) # TODO Question : why is there no overlapping result with random.split(key) (so only one number generated with the same key)
key, subkeys

(array([3306097435, 3899823266], dtype=uint32),
 [array([147607341, 367236428], dtype=uint32),
  array([2280136339, 1907318301], dtype=uint32),
  array([ 781391491, 1939998335], dtype=uint32)])

You can think about those PRNGs as trees of keys that matches the structure of your models, which is important for reproducibility and soundness of the random behavior that you expect.

## Gradients and autodiff

For a full overview, you can have a look at JAX's documentation on Autodiff [here](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html).

**TODO: add something about the computational cost vjp vs jvp.**

### Gradients

JAX provides first-class support for gradients and automatic differentiation in functions. This is also where the functional paradigm shines, since gradients on functions are essentially stateless operations. If we consider a simple function $f:\mathbb{R}^n\rightarrow\mathbb{R}$

$$f(x) = \frac{1}{2} x^T x$$
with the (known) gradient:
$$\nabla f(x) = x$$

In [ ]:
key = random.PRNGKey(0)
def f(x):
  return jnp.dot(x.T,x)/2.0

v = random.normal(key,(4,))
f(v)

DeviceArray(2.1347964, dtype=float32)

JAX computes the gradient as an operator acting on functions with `jax.grad`. Note that this only works for scalar valued functions.

Let's take the gradient of f and make sure it matches the identity map.

In [ ]:
v = random.normal(key,(4,))
print("Original v:")
print(v)
print("Gradient of f taken at point v")
print(jax.grad(f)(v)) # should be equal to v !

Original v:
[ 1.8160858 -0.7548852  0.339889  -0.5348355]
Gradient of f taken at point v
[ 1.8160858 -0.7548852  0.339889  -0.5348355]


As mentionned, previously, `jax.grad` only works for scalar valued functions. JAX can also handle general vector valued functions. The most useful primitives are the so-called jacobian vector product `jax.jvp` and vector jacobian product `jax.vjp`.

### Jacobian vector product

Let's consider a map $f:\mathbb{R}^n\rightarrow\mathbb{R}^m$. As a reminder, the differential of f is the map $df:\mathbb{R}^n \rightarrow \mathcal{L}(\mathbb{R}^n,\mathbb{R}^m)$ where $\mathcal{L}(\mathbb{R}^n,\mathbb{R}^m)$ is the space of linear maps from $\mathbb{R}^n$ to $\mathbb{R}^m$ (hence $df(x)$ is often represented as a Jacobian matrix). The linear approximation of f at point $x$ reads:
$$f(x+v) = f(x) + df(x)\bullet v + o(v)$$
The $\bullet$ operator means you are applying the linear map $df(x)$ to the vector v.

Even though you are rarely interested in computing the full Jacobian matrix representing the linear map $df(x)$ in a standard basis, you are often interested in the quantity $df(x)\bullet v$. This is exactly what `jax.jvp` is for, and `jax.jvp(f, (x,), (v,))` returns the tuple:
$$(f(x), df(x)\bullet v)$$


Let's use a simple function as an example: $f(x) = \frac{1}{2}({x_1}^2, {x_2}^2, \ldots, {x_n}^2)$ where we know that $df(x)\bullet h = (x_1h_1, x_2h_2,\ldots,x_nh_n)$. Hence using `jax.jvp` with $h= (1,1,\ldots,1)$ should return $x$ as an output.

In [ ]:
def f(x):
  return jnp.multiply(x,x)/2.0

x = random.normal(key, (5,))
v = jnp.ones(5)
print("(x,f(x))")
print((x,f(x)))
print("jax.jvp(f, (x,),(v,))")
print(jax.jvp(f, (x,),(v,)))

(x,f(x))
(DeviceArray([ 0.18784378, -1.2833427 , -0.27109176,  1.2490592 ,
              0.24446994], dtype=float32), DeviceArray([0.01764264, 0.82348424, 0.03674537, 0.7800744 , 0.02988278],            dtype=float32))
jax.jvp(f, (x,),(v,))
(DeviceArray([0.01764264, 0.82348424, 0.03674537, 0.7800744 , 0.02988278],            dtype=float32), DeviceArray([ 0.18784378, -1.2833427 , -0.27109176,  1.2490592 ,
              0.24446994], dtype=float32))


### Vector Jacobian product
Keeping our $f:\mathbb{R}^n\rightarrow\mathbb{R}^m$ it's often the case (for example when you are working with a scalar loss function) that you are interested in the composition $x\rightarrow\phi\circ f(x)$ where $\phi \in \mathcal{L}(\mathbb{R}^m,\mathbb{R})$ (dual space of $\mathbb{R}^m$). In that case, the gradient reads:
$$\nabla(\phi\circ f)(x) = J_f(x)^T\nabla\phi(f(x))$$
Where $J_f(x)$ is the Jacobian matrix of f evaluated at x, meaning that $df(x)\bullet v = J_f(x)v$.

`jax.vjp(f,x)` returns the tuple:
$$(f(x),v\rightarrow v^TJ_f(x))$$

Keeping the same example as previously, using $v=(1,\ldots,1)$, applying the VJP function returned by JAX should return the $x$ value:

In [ ]:
(val, jvp_fun) = jax.vjp(f,x)
print("x = ", x)
print("v^T Jf(x) = ", jvp_fun(jnp.ones((5,)))[0])

x =  [ 0.18784378 -1.2833427  -0.27109176  1.2490592   0.24446994]
v^T Jf(x) =  [ 0.18784378 -1.2833427  -0.27109176  1.2490592   0.24446994]


## Accelerating code with jit & ops vectorization
We borrow the following example from the [JAX quickstart](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html).

---

### Jit

JAX uses the XLA compiler under the hood, and enables you to jit compile your code to make it faster and more efficient. This is the purpose of the @jit annotation. 

In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

v = random.normal(key, (1000000,))
%timeit selu(v).block_until_ready()

The slowest run took 20.23 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 6.48 ms per loop


Now using the jit annotation (or function here) to speed things up:

In [ ]:
selu_jit = jax.jit(selu)
%timeit selu_jit(v).block_until_ready()

The slowest run took 24.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.56 ms per loop


jit compilation can be used along with autodiff in the code transparently.

---
### Vectorization

Finally, JAX enables you to write code that apply to a single example, and then vectorize it to manage transparently batching dimensions.

In [ ]:
mat = random.normal(key, (15, 10))
batched_x = random.normal(key, (5, 10)) # Batching on first dimension
single = random.normal(key, (10,))

def apply_matrix(v):
  return jnp.dot(mat, v)

print("Single apply shape: ", apply_matrix(single).shape)
print("Batched example shape: ", jax.vmap(apply_matrix)(batched_x).shape)

Single apply shape:  (15,)
Batched example shape:  (5, 15)


## Full example: linear regression

Let's implement one of the simplest model using everything we saw so far: linear regression. From a set of data points $\{(x_i,y_i), i\in \{1,\ldots, k\}, x_i\in\mathbb{R}^n,y_i\in\mathbb{R}^m\}$, we try to find a set of parameters $W\in \mathcal{M}_{m,n}(\mathbb{R}), b\in\mathbb{R}^m$ such that the function $f_{W,b}(x)=Wx+b$ minimizes the mean squared error:
$$\mathcal{L}(W,b)\rightarrow\frac{1}{k}\sum_{i=1}^{k} \frac{1}{2}\|y_i-f_{W,b}(x_i)\|^2_2$$
(Note: this is a rough explanation, theoretically we should be minimizing the expectation of the loss, however for the sake of simplicity here we consider only the sampled loss).

In [ ]:
key = random.PRNGKey(0)

# Create the predict function from a set of parameters
def make_predict(W,b):
  def predict(x):
    return jnp.dot(W,x)+b
  return predict

# Create the loss from the data points set
def make_mse(x_batched,y_batched):
  def mse(W,b):
    # Define the squared loss for a single pair (x,y)
    def squared_error(x,y):
      return jnp.inner(y-jnp.dot(W,x)-b,y-jnp.dot(W,x)-b)/2.0
    # We vectorize the previous to compute the average of the loss on all samples.
    return jnp.mean(jax.vmap(squared_error)(x_batched,y_batched), axis=0)
  return jax.jit(mse) # And finally we jit the result.

In [ ]:
# Set problem dimensions
nsamples = 20
xdim = 10
ydim = 5

# Generate random ground truth W and b
k1, k2 = random.split(key)
W = random.normal(k1, (ydim, xdim))
b = random.normal(k2, (ydim,))
true_predict = make_predict(W,b)

# Generate samples with additional noise
ksample, knoise = random.split(k1)
x_samples = random.normal(ksample, (nsamples, xdim))
y_samples = jax.vmap(true_predict)(x_samples) + 0.1*random.normal(knoise,(nsamples, ydim))

# Generate MSE for our samples
mse = make_mse(x_samples,y_samples)

In [ ]:
# Initialize estimated W and b with zeros.
What = jnp.zeros_like(W)
bhat = jnp.zeros_like(b)

alpha = 0.3 # Gradient step size
print('Loss for "true" W,b: ', mse(W,b))
for i in range(101):
  # We perform one gradient update
  What, bhat = What - alpha*jax.grad(mse,0)(What,bhat), bhat - alpha*jax.grad(mse,1)(What,bhat)
  if (i%5==0):
    print("Loss step {}: ".format(i), mse(What,bhat))

Loss for "true" W,b:  0.023639774
Loss step 0:  11.096583
Loss step 5:  1.1743387
Loss step 10:  0.3287935
Loss step 15:  0.13981776
Loss step 20:  0.073595665
Loss step 25:  0.04415299
Loss step 30:  0.029408697
Loss step 35:  0.021554668
Loss step 40:  0.017227937
Loss step 45:  0.014798884
Loss step 50:  0.013420247
Loss step 55:  0.012632713
Loss step 60:  0.0121811
Loss step 65:  0.011921472
Loss step 70:  0.011771994
Loss step 75:  0.01168585
Loss step 80:  0.011636159
Loss step 85:  0.011607497
Loss step 90:  0.011590948
Loss step 95:  0.011581404
Loss step 100:  0.011575884


This is obviously an approximate solution to the linear regression problem (solving it would require a bit more work!), but here you have all the tools you would need if you wanted to do it the proper way.

## Refining a bit with pytrees

Here we're going to elaborate on our previous example using JAX pytree datastructure.

### Pytrees basics

The JAX ecosystem uses pytrees everywhere and we do as well in Flax (the previous FrozenDict example is one, we'll get back to this). For a complete overview, we suggest that you take a look at the [pytree page](https://jax.readthedocs.io/en/latest/pytrees.html) from JAX's doc:

*In JAX, a pytree is a container of leaf elements and/or more pytrees. Containers include lists, tuples, and dicts (JAX can be extended to consider other container types as pytrees, see Extending pytrees below). A leaf element is anything that’s not a pytree, e.g. an array. In other words, a pytree is just a possibly-nested standard or user-registered Python container. If nested, note that the container types do not need to match. A single “leaf”, i.e. a non-container object, is also considered a pytree.*

```python
[1, "a", object()] # 3 leaves

(1, (2, 3), ()) # 3 leaves

[1, {"k1": 2, "k2": (3, 4)}, 5] # 5 leaves
```

JAX provides a few utilities to work with pytrees that lives in the `tree_util` package.

In [ ]:
from jax import tree_util

t = [1, {"k1": 2, "k2": (3, 4)}, 5]

You will often come across `tree_map` function that maps a function f to a tree and its leaves. We used it in the previous section to display the shapes of the model's parameters.

In [ ]:
tree_util.tree_map(lambda x: x*x, t)

[1, {'k1': 4, 'k2': (9, 16)}, 25]

A more flexible version of `tree_map` would be `tree_multimap`. Instead of applying a standalone function to each of the tree leafs, you also provide a tuple of additional trees with similar shape to the input tree that will provide per leaf arguments to the function.

In [ ]:
tree_util.tree_multimap(lambda x,y: x+y, t, tree_util.tree_map(lambda x: x*x, t))

[2, {'k1': 6, 'k2': (12, 20)}, 30]

### Linear regression with Pytrees

Whereas our previous example was perfectly fine, we can see that when things get more complicated (as they will with neural networks), it will be harder to manage parameters of the models as we did. Here we show an alternative based on pytrees.

In [ ]:
key = random.PRNGKey(0)

# Create the predict function from a set of parameters
def make_predict_pytree(params):
  def predict(x):
    return jnp.dot(params['W'],x)+params['b']
  return predict

# Create the loss from the data points set
def make_mse_pytree(x_batched,y_batched):
  def mse(params):
    # Define the squared loss for a single pair (x,y)
    def squared_error(x,y):
      return jnp.inner(y-jnp.dot(params['W'],x)-params['b'],y-jnp.dot(params['W'],x)-params['b'])/2.0
    # We vectorize the previous to compute the average of the loss on all samples.
    return jnp.mean(jax.vmap(squared_error)(x_batched,y_batched), axis=0)
  return jax.jit(mse) # And finally we jit the result.

Our `params` is a pytree containing both the `W` and `b` entries. We now generate our loss using pytree params. The great thing is that JAX is able to handle differentiation with respect to pytree parameters:

In [ ]:
# Generate MSE for our samples
mse_pytree = make_mse_pytree(x_samples,y_samples)

# Initialize estimated W and b with zeros.
params = {'W': jnp.zeros_like(W), 'b': jnp.zeros_like(b)}

jax.grad(mse_pytree)(params)

{'W': DeviceArray([[-1.9287349e+00,  4.2963740e-01,  7.1613449e-01,
                2.1056123e+00,  5.0405109e-01, -2.4983375e+00,
               -6.3854158e-01, -2.2620211e+00, -1.3365203e+00,
               -2.0426038e-01],
              [ 1.1999468e+00, -9.4563609e-01, -1.0878401e+00,
               -7.0340693e-01,  3.3224618e-01,  1.7538788e+00,
               -7.1916533e-01,  1.0927426e+00, -1.4491035e+00,
                5.9715635e-01],
              [-1.4826508e+00, -7.6116526e-01,  2.2319850e-01,
               -3.0391920e-01,  3.0397053e+00, -3.8419446e-01,
               -1.8290073e+00, -2.3353369e+00, -1.1087126e+00,
               -7.7454001e-01],
              [ 8.2374448e-01, -9.9650615e-01, -7.6030099e-01,
                6.3919234e-01, -6.0864888e-02, -1.0859717e+00,
                1.2923399e+00, -4.9342909e-01, -1.4710285e-03,
                1.2977618e+00],
              [-4.5656443e-01, -1.3063036e-01, -3.9178997e-01,
                2.1743820e+00, -5.3948764e-02,  

Now using our tree of params, we can write the gradient descent in a simpler way using `jax.tree_multimap`:

In [ ]:
alpha = 0.3 # Gradient step size
print('Loss for "true" W,b: ', mse_pytree({'W':W,'b':b}))
for i in range(101):
  # We perform one gradient update
  params = jax.tree_multimap(lambda old,grad: old-alpha*grad, params, jax.grad(mse_pytree)(params))
  if (i%5==0):
    print("Loss step {}: ".format(i), mse_pytree(params))

Loss for "true" W,b:  0.023639774
Loss step 0:  11.096583
Loss step 5:  1.1743387
Loss step 10:  0.3287935
Loss step 15:  0.13981776
Loss step 20:  0.073595665
Loss step 25:  0.04415299
Loss step 30:  0.029408697
Loss step 35:  0.021554668
Loss step 40:  0.017227937
Loss step 45:  0.014798884
Loss step 50:  0.013420247
Loss step 55:  0.012632713
Loss step 60:  0.0121811
Loss step 65:  0.011921472
Loss step 70:  0.011771994
Loss step 75:  0.01168585
Loss step 80:  0.011636159
Loss step 85:  0.011607497
Loss step 90:  0.011590948
Loss step 95:  0.011581404
Loss step 100:  0.011575884


That's all you needed to know to get started with Flax! To dive deeper, we very much recommend to check JAX [docs](https://jax.readthedocs.io/en/latest/index.html).